<a href="https://colab.research.google.com/github/OneSll/Age_gender_recognition/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from io import BytesIO
import pickle

In [ ]:
import torch.nn as nn
from torchvision import models

# Определение модели ResNet-18
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)

        # Размораживаем последние три слоя
        for param in list(self.resnet.layer4.parameters()) + \
                     list(self.resnet.layer3.parameters()):
            param.requires_grad = True

        # Заменяем последний полносвязный слой на новый слой с количеством выходных классов
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

In [ ]:
from sklearn.preprocessing import LabelEncoder


labels = [
    'Женщина средних лет с вероятностью',
    'Ребенок мужского пола с вероятностью',
    'Пожилой мужчина с вероятностью',
    'Мужчина средних лет с вероятностью',
    'Пожилая женщина с вероятностью',
    'Ребенок женского пола с вероятностью',
]
label_encoder = LabelEncoder()
label_encoder.fit(labels)

def predict_one_sample(model, image_bytes,):
    """Predict for a single image."""
     # Convert image bytes to PIL Image
    image = Image.open(BytesIO(image_bytes)).convert('RGB')

    # Define transformation
    transform = transforms.Compose([
        transforms.Resize((224, 224)),   # Resize image to the expected size
        transforms.ToTensor(),           # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])
    inputs = transform(image).unsqueeze(0)

    # Perform the prediction
    with torch.no_grad():
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()

    predicted_proba = np.max(probs)*100
    y_pred = np.argmax(probs)

    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label, predicted_proba)

    return predicted_text


In [ ]:
label_encoder.classes_

In [ ]:
# Load the complete model
weights = '/content/drive/MyDrive/model_weights.pth'

model = CustomResNet(num_classes=6)
model.load_state_dict(torch.load(weights, map_location=torch.device('cpu')))

In [ ]:
import telebot

# Initialize the Telegram bot
bot = telebot.TeleBot("6265291697:AAEhkyI-muYJG9aTXYqHtyxzOAWoEEKkbOg")

@bot.message_handler(commands=['start'])
def start_message(message):
    welcome_text = (
            "Привет! Я бот для распознавания персонажей из мультипликационного сериала 'Симпсоны'. "
            "Загрузите изображение персонажа, и я определю его пол и возрастную группу.\n\n"
            "Я могу распознать следующие категории:\n"
            "1. Мужчина средних лет\n"
            "2. Ребенок мужского пола\n"
            "3. Пожилой мужчина\n"
            "4. Женщина средних лет\n"
            "5. Ребенок женского пола\n"
            "6. Пожилая женщина\n\n"
            "Пожалуйста, загрузите изображение, чтобы начать."
        )
    bot.send_message(message.chat.id, welcome_text)

@bot.message_handler(content_types=['photo'])
def process_image(message):
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    file_path = file_info.file_path

    # Download the image
    downloaded_file = bot.download_file(file_path)

    # Perform prediction
    predictions = predict_one_sample(model, downloaded_file)

    # Convert prediction results to a readable format
    response = "Результаты предсказания:\n"

    bot.reply_to(message, predictions)

# Start the bot
bot.infinity_polling(none_stop=True, interval=1)
